# Generating participant hierarchies

This notebook provides the tools for organizing the participants of a chapter into a semantic hierarchy based on synonymous and part-whole relationships. This is the first step for analyzing the participants of a text. Another notebook provides measures for collocating and disambiguating participants across a span of chapters (cf. "1b_Disambiguating and Collocating Participants.ipynb").

In [1]:
import os, sys, re
from collections import defaultdict
from anytree.exporter import DotExporter
from anytree import Node, RenderTree, findall, findall_by_attr, find_by_attr
import graphviz

In [2]:
#Specifying path to graphviz
os.environ["PATH"] += os.pathsep + 'C:/Users/Ejer/Anaconda3/Library/bin/graphviz'

In [3]:
from tf.app import use
A = use('bhsa', hoist=globals(), mod='ch-jensen/participants/actor/tf')

In [4]:
import Nodes

Ancillary functions:

In [5]:
def show(actor,allrefs,n=int()):
    '''
    This function shows all references pertaining to a particular participant label
    '''
    if not n:
        node = allrefs[actor][0]
    else:
        node = n
    A.pretty(L.u(node, 'verse')[0], highlights = {node: 'red'})
    
def checkList(allrefs):
    '''
    Simple ancillary function to check whether an actor has only one reference which is either a Pred, PreC, or PtcO.
    This function is useful because single-case actors are not normally included in the participant data.
    '''
    
    for actor in allrefs:
        if len(allrefs[actor]) == 1:
            ph = L.u(allrefs[actor][0], 'phrase')[0]
            func = F.function.v(ph)
            if func in {'Pred','PreC','PtcO'}:
                print(actor)

Functions for creating trees:

In [5]:
def edgetypefunc(node, child):
    return '--'

def createPNG(tree, book, chapter):
    title = f'trees/ParticipantTreeStructure_{book[:3]}_{chapter}.png'
    DotExporter(tree, graph="digraph").to_picture(title)
    
def createPNG_extract(tree, book, chapter, node, max_indentation):
    title = f'trees/ParticipantTreeStructure_subtree_{book[:3]}_{chapter}.png'
    subtree = find_by_attr(tree, node, maxlevel=max_indentation)
    DotExporter(subtree, graph="digraph").to_picture(title)    

## Lev 17

#### a. Listing all participants and their references

In [7]:
Lev17 = Nodes.GenerateNodes('Leviticus',17)
#GenerateNodes('Leviticus',17).actorDict()
Lev17.allRefs()
Lev17.AllRefs

{'JHWH': [943175,
  943176,
  943178,
  943187,
  943188,
  943189,
  943207,
  1317274,
  943228,
  943234,
  1317293,
  943246,
  943279,
  943295,
  63229,
  943302,
  943310,
  943311,
  943322,
  943360],
 'MCH=': [943177, 943179, 943182, 943264],
 '>HRN BN ->HRN KL BN JFR>L': [943180, 63028],
 '>HRN': [1317252, 63020],
 'BN JFR>L': [1317258,
  943219,
  943220,
  63100,
  943223,
  943224,
  943227,
  943232,
  943249,
  63153,
  943254,
  943255,
  63165,
  63167,
  63169,
  63255,
  943314,
  63262,
  943323,
  63279,
  63289,
  1317334,
  63306,
  943361,
  943364],
 'JFR>L': [1317257,
  1317263,
  1317283,
  1317305,
  1317317,
  1317327,
  1317333,
  1317345],
 'DBR': [943184, 943185],
 '>JC >JC': [943190,
  943193,
  943198,
  943204,
  943205,
  943211,
  943213,
  943215,
  943216,
  63093],
 '>JC': [1317261, 943285, 943286, 1317331],
 'BJT JFR>L': [943191, 1317306, 63183, 1317318, 63222],
 'CWR KFB <Z==': [943194, 63066],
 'MXNH': [943195, 943200],
 'PTX >HL MW<D': [9432

* '>JC >JC refers to a person belonging to the house of Israel
* '>JC >JC#2 refers to a person belonging to the house of Israel or a sojourner.

#### Identifying participants to include (here human/divine participants)

In [9]:
humanRefs_Lev17 = ['JHWH', 'MCH=', '>HRN BN ->HRN KL BN JFR>L', '>HRN', 'BN JFR>L', '>JC >JC', '>JC', 'BJT JFR>L', 'KHN',
                   '>JC >JC#2', 'BJT JFR>L GR', 'GR', 'NPC', 'NPC#2', 'KL', 'MN <M/ ->JC >JC','MN QRB/ <M/ -NPC',
                   'MN QRB/ <M/ ->JC >JC','KL/ NPC/','F<JR=','B TWK/ -BJT JFR>L','B TWK/ -BJT JFR>L#2']

#### Organizing the participants into synonyms or hyponyms (part-whole relations)

This task is manual and obviously somewhat subjective.

In [10]:
synonyms_Lev17 = {'>JC >JC': ['>JC'], 'KL/ NPC/': ['NPC#2'], '>JC >JC#2': ['NPC', 'KL'],
                  'BJT JFR>L':['B TWK/ -BJT JFR>L','B TWK/ -BJT JFR>L#2'], 'MN <M/ ->JC >JC':['MN QRB/ <M/ -NPC',
                                                                                              'MN QRB/ <M/ ->JC >JC']}

hyponyms_Lev17 = {'>HRN BN ->HRN KL BN JFR>L': ['>HRN', 'BN JFR>L'],'BJT JFR>L GR': ['BJT JFR>L','GR'],
                  '>JC >JC#2':['>JC >JC','GR']}

In [11]:
#show('B TWK/ -BJT JFR>L',Lev17.AllRefs,943230)

A tree is created based on the synonyms and hyponyms defined:

In [12]:
tree = Lev17.tree(humanRefs_Lev17, synonyms_Lev17, hyponyms_Lev17)

In [14]:
Lev17.treeStructure()

participants
├── >HRN BN ->HRN KL BN JFR>L
│   ├── >HRN
│   └── BN JFR>L
├── BJT JFR>L GR
│   ├── BJT JFR>L
│   │   ├── B TWK/ -BJT JFR>L
│   │   │   └── BJT JFR>L
│   │   └── B TWK/ -BJT JFR>L#2
│   │       └── BJT JFR>L
│   └── GR
├── >JC >JC#2
│   ├── >JC >JC
│   │   └── >JC
│   │       └── >JC >JC
│   ├── GR
│   ├── NPC
│   │   └── >JC >JC#2
│   └── KL
│       └── >JC >JC#2
├── KL/ NPC/
│   └── NPC#2
│       └── KL/ NPC/
├── MN <M/ ->JC >JC
│   ├── MN QRB/ <M/ -NPC
│   │   └── MN <M/ ->JC >JC
│   └── MN QRB/ <M/ ->JC >JC
│       └── MN <M/ ->JC >JC
├── JHWH
├── MCH=
├── KHN
└── F<JR=


The tree can be exported:

In [6]:
createPNG(tree, 'Leviticus', 17)

NameError: name 'tree' is not defined

A subset of the tree can be exported:

In [ ]:
#createPNG_extract(tree, 'Leviticus', 17, node='>JC >JC#2', max_indentation=2)

Finally, the resulting list of participants and their references can be generated:

In [16]:
Lev17.resultRefs()

defaultdict(list,
            {'JHWH': [943175,
              943176,
              943178,
              943187,
              943188,
              943189,
              943207,
              1317274,
              943228,
              943234,
              1317293,
              943246,
              943279,
              943295,
              63229,
              943302,
              943310,
              943311,
              943322,
              943360],
             'MCH=': [943177, 943179, 943182, 943264],
             '>HRN BN ->HRN KL BN JFR>L': [943180, 63028],
             '>HRN': [1317252, 63020, 943180, 63028],
             'BN JFR>L': [1317258,
              943219,
              943220,
              63100,
              943223,
              943224,
              943227,
              943232,
              943249,
              63153,
              943254,
              943255,
              63165,
              63167,
              63169,
              63255,
     

Compare resulting list with initial list of all references:

In [17]:
error_list_Lev17 = Lev17.checkResults()
print(len(error_list_Lev17))

0


## Lev 18

In [18]:
Lev18 = Nodes.GenerateNodes('Leviticus',18)

Lev18.allRefs()
Lev18.AllRefs

{'JHWH': [943401,
  943402,
  943404,
  943410,
  943411,
  943412,
  943422,
  943423,
  63436,
  63440,
  943439,
  943440,
  943441,
  63451,
  63454,
  943452,
  943453,
  943460,
  943461,
  943574,
  943575,
  943606,
  943607,
  943613,
  63724,
  63727,
  63804,
  943676,
  943677,
  943678],
 'MCH=': [943403, 943405, 943408],
 'BN JFR>L': [943406,
  63405,
  63408,
  943415,
  943418,
  63426,
  943427,
  943431,
  943433,
  943436,
  943437,
  63447,
  943443,
  943457,
  943458,
  943599,
  63705,
  943621,
  943622,
  943626,
  63745,
  63759,
  63769,
  943646,
  63771,
  63781,
  943665,
  943667,
  63815,
  943674,
  63822],
 '>RY MYRJM': [1317365, 63415],
 '>RY KN<N': [1317369, 63432],
 'MCPV -JHWH': [943432, 63444, 1317371, 1317473],
 'XQH -JHWH': [943435, 1317370, 1317472],
 'XQH -JHWH MCPV -JHWH': [943444, 63457, 63462],
 '>DM': [943446, 943448, 943450],
 '>JC >JC': [943454, 63470],
 '>B -2ms': [1317379,
  1317386,
  1317389,
  1317393,
  1317415,
  1317419,
  131742

In [19]:
humanRefs_Lev18 = ['JHWH','MCH=','BN JFR>L','>DM','>JC >JC','>B -2ms','>M -2ms','2ms','<RWH >CH >B -2ms','>CH >B -2ms',
                  'BT >B -2msBT >M -2ms','BN -2ms','C>R >B -2ms','C>R >M -2ms','>X >B -2ms','DWDH -2ms',
                   '>CH BN -2ms','<RWH >CH >X -2ms','>X -2ms','>CH','C>RH','>XWT ->CH','>CH <MJT -2ms','GWJ','GR','>JC >RY',
                   'KL','NPC','>L KL/ C>R/ BFR/ ->JC >JC','<RWH/ >B/ -2msW <RWH/ >M/ -2ms','<RWH/ ->M -2ms','<RWH/ -',
                   '<RWH/ -<RWH -2ms','<RWH/ -#2','<RWH/ >XWT/ >B/ -2ms','<RWH/ >XWT/ >M/ -2ms','<RWH/ >X/ >B/ -2ms',
                   '<RWH/ KLH/ -2ms','<RWH/ ->CH BN -2ms','<RWH/ >CH/ W BT/ ->CH','>T BT/ BN/ ->CH W >T BT/ BT/ ->CH',
                   '<RWH/ ->CH','<RWH/ ->CH#2','<RWH/ ->CH#3','MN ZR</ -2ms','L H MLK=/', '>T CM/ >LHJM/ -2ms','>T== ZKR=/',
                   '>T JCB[ ->RY','MN QRB/ <M/ -NPC','L PNH/ -BN JFR>L','MN PNH/ -BN JFR>L','B TWK/ -BN JFR>L']

landRefs_Lev18 = ['>RY MYRJM','>RY KN<N','>RY']

In [20]:
synonyms_Lev18 = {'JHWH': ['>T CM/ >LHJM/ -2ms'],
                  'BN JFR>L': ['>JC >JC','L PNH/ -BN JFR>L','MN PNH/ -BN JFR>L','B TWK/ -BN JFR>L'],
                  '>JC >RY':['GWJ','>T JCB[ ->RY'], 'NPC': ['KL'], '>M -2ms':['<RWH/ ->M -2ms'],
                  'BT >B -2msBT >M -2ms':['<RWH/ -', '<RWH/ -#2'], '<RWH/ >XWT/ >B/ -2ms': ['C>R >B -2ms'],
                  '>CH >B -2ms':['<RWH >CH >B -2ms'],'<RWH/ >XWT/ >M/ -2ms': ['C>R >M -2ms'],
                 'DWDH -2ms': ['<RWH/ >X/ >B/ -2ms'], '>CH BN -2ms': ['<RWH/ KLH/ -2ms', '<RWH/ ->CH BN -2ms'],
                  '>CH': ['<RWH/ ->CH', '<RWH/ ->CH#3','<RWH/ ->CH#2']}

hyponyms_Lev18 = {'>L KL/ C>R/ BFR/ ->JC >JC': ['<RWH/ >B/ -2msW <RWH/ >M/ -2ms','>CH >B -2ms',
                                                                    'BT >B -2msBT >M -2ms','<RWH/ -<RWH -2ms',
                  '<RWH/ >XWT/ >B/ -2ms','<RWH/ >XWT/ >M/ -2ms','DWDH -2ms','>CH BN -2ms','<RWH >CH >X -2ms'],
                  '<RWH/ >CH/ W BT/ ->CH': ['>CH'], 'C>RH': ['<RWH/ >CH/ W BT/ ->CH','>T BT/ BN/ ->CH W >T BT/ BT/ ->CH'],
                  '<RWH/ >B/ -2msW <RWH/ >M/ -2ms': ['>B -2ms','>M -2ms']}

In [ ]:
show('>JC',Lev18.AllRefs)

#### Notes:
* '<RWH' refers to "nakedness" but also to the body of a person, therefore included as referring to the person.
* '>DM: Generic, 3ms. Relates to the addressees but as a general principle for how human beings should live (v. 5). This generic term for all human beings is interpreted as synonymous to KL and NPC (v. 29).
* '>JC >JC: Another generic term but clearly means "anyone of you", that is, a member of the addresses (synonym) (cf. 18:6).
* A change between plural and singular referenc in 18:7. Here, the two references are differentiated.
* KL: Is KL synonymous to "you" (pl.)?
* '>L KL/ C>R/ BFR/ ->JC >JC: Is a collective for all female relatives in the subsequent text (v. 6).
* 'MN ZR</ -2ms': Refers to both "your daughter" and "your son", but not the other way round.
* C>RH: Is a hypernym of >CH and her grandchildren (v. 17).
* '>CH' refers to a woman to which the person is not related. This reference (along with references to the woman's dauther and reference to the wife a fellow) is therefore not a logical hyponym of the reference in v. 6.

In [21]:
tree = Lev18.tree(humanRefs_Lev18, synonyms_Lev18, hyponyms_Lev18)

In [22]:
Lev18.treeStructure()
createPNG(tree, 'Leviticus', 18)

participants
├── >L KL/ C>R/ BFR/ ->JC >JC
│   ├── <RWH/ >B/ -2msW <RWH/ >M/ -2ms
│   │   ├── >B -2ms
│   │   └── >M -2ms
│   │       └── <RWH/ ->M -2ms
│   │           └── >M -2ms
│   ├── >CH >B -2ms
│   │   └── <RWH >CH >B -2ms
│   │       └── >CH >B -2ms
│   ├── BT >B -2msBT >M -2ms
│   │   ├── <RWH/ -
│   │   │   └── BT >B -2msBT >M -2ms
│   │   └── <RWH/ -#2
│   │       └── BT >B -2msBT >M -2ms
│   ├── <RWH/ -<RWH -2ms
│   ├── <RWH/ >XWT/ >B/ -2ms
│   │   └── C>R >B -2ms
│   │       └── <RWH/ >XWT/ >B/ -2ms
│   ├── <RWH/ >XWT/ >M/ -2ms
│   │   └── C>R >M -2ms
│   │       └── <RWH/ >XWT/ >M/ -2ms
│   ├── DWDH -2ms
│   │   └── <RWH/ >X/ >B/ -2ms
│   │       └── DWDH -2ms
│   ├── >CH BN -2ms
│   │   ├── <RWH/ KLH/ -2ms
│   │   │   └── >CH BN -2ms
│   │   └── <RWH/ ->CH BN -2ms
│   │       └── >CH BN -2ms
│   └── <RWH >CH >X -2ms
├── <RWH/ >CH/ W BT/ ->CH
│   └── >CH
│       ├── <RWH/ ->CH
│       │   └── >CH
│       ├── <RWH/ ->CH#3
│       │   └── >CH
│       └── <RWH/ ->CH#2
│     

In [23]:
Lev18.resultRefs()

defaultdict(list,
            {'JHWH': [943401,
              943402,
              943404,
              943410,
              943411,
              943412,
              943422,
              943423,
              63436,
              63440,
              943439,
              943440,
              943441,
              63451,
              63454,
              943452,
              943453,
              943460,
              943461,
              943574,
              943575,
              943606,
              943607,
              943613,
              63724,
              63727,
              63804,
              943676,
              943677,
              943678,
              943573],
             'MCH=': [943403, 943405, 943408],
             'BN JFR>L': [943406,
              63405,
              63408,
              943415,
              943418,
              63426,
              943427,
              943431,
              943433,
              943436,
              943437,


In [24]:
error_list_Lev18 = Lev18.checkResults()
print(len(error_list_Lev18))

0


## Lev 19

In [25]:
Lev19 = Nodes.GenerateNodes('Leviticus',19)
Lev19.allRefs()
Lev19.AllRefs

{'JHWH': [943680,
  943681,
  943683,
  943692,
  943693,
  943694,
  943695,
  63853,
  943702,
  943703,
  943704,
  943713,
  943714,
  943715,
  943720,
  943736,
  1317513,
  943775,
  943776,
  943777,
  63983,
  943796,
  943797,
  943820,
  943821,
  943843,
  943844,
  943867,
  943868,
  64086,
  943915,
  1317553,
  943964,
  943965,
  943966,
  943993,
  943994,
  64273,
  64276,
  944012,
  944013,
  944023,
  944024,
  944025,
  944034,
  944035,
  944061,
  944062,
  944063,
  944073,
  944074,
  944075,
  944077,
  64384,
  64388,
  944086,
  944087],
 'MCH=': [943682, 943684, 943687],
 'GR': [1317498,
  63838,
  943689,
  943690,
  63845,
  943698,
  943701,
  63857,
  943706,
  943711,
  63868,
  63871,
  943718,
  63880,
  943722,
  63884,
  943754,
  63932,
  63935,
  943759,
  63967,
  943779,
  943782,
  943785,
  943790,
  943823,
  943870,
  943935,
  943938,
  943941,
  64187,
  943959,
  943961,
  64215,
  64219,
  943968,
  943971,
  943974,
  943976,
  64233

#### Notes:

* Human references governed by L PNH ("in front of") are included, because the construction functions like a preposition.
* The soul (NPC) in v. 28 is dubious. Most likely, it does not denote a soul in general, cf. the other uses of NPC in the chapter.
* GR refers to BN JFR>L in this chapter, but since GR is the first full phrase referring to that referent, it is used as the label.
* MWT[ refers to the man and the slave girl having sexual intercourse
* SLX[ refers to the man who is being forgiven (a passive)

In [26]:
humanRefs_Lev19 = ['JHWH','MCH=','GR','>JC','3ms','NPC','2ms','GR#2','>LHJM -2ms','R<= -2ms','<MJT -2ms','CPXH','KHN',
                  'BT -2ms','JD<NJ','MYRJM','>L KL/ <DH/ BN/ JFR>L/','>M/ ->JC W >B/ ->JC','>L H >LJL/','MN <M/ -NPC',
                   'L H <NJ/ W L H GR/', 'B CM/ -JHWH','>T CM/ >LHJM/ -2ms','XRC=/','PNH/ DL/','PNH/ GDWL/',
                   'B <MJT/ ->JC','L PNH/ <WR/','B <M/ -2ms','>T >X/ -2ms','>T BN/ <M/ -2ms','L >JC/',
                   'MWT[','SLX[','>L H >WB/','MN PNH/ FJBH/','PNH/ ZQN/']

land_references_Lev19 = ['>RY -GR','>RY','>RY MYRJM']

In [27]:
synonyms_Lev19 = {'JHWH': ['>LHJM -2ms','B CM/ -JHWH','>T CM/ >LHJM/ -2ms',], 'GR': ['>L KL/ <DH/ BN/ JFR>L/'],
                  '3ms':['NPC'], '>JC':['SLX[']}

hyponyms_Lev19 = {'L H <NJ/ W L H GR/': ['GR#2'], 'MWT[':['>JC','CPXH']}

In [ ]:
show('JD<NJ',Lev19.AllRefs)

In [28]:
tree = Lev19.tree(humanRefs_Lev19, synonyms_Lev19, hyponyms_Lev19)

In [29]:
Lev19.treeStructure()
createPNG(tree, 'Leviticus', 19)

participants
├── L H <NJ/ W L H GR/
│   └── GR#2
├── MWT[
│   ├── >JC
│   │   └── SLX[
│   │       └── >JC
│   └── CPXH
├── JHWH
│   ├── >LHJM -2ms
│   │   └── JHWH
│   ├── B CM/ -JHWH
│   │   └── JHWH
│   └── >T CM/ >LHJM/ -2ms
│       └── JHWH
├── GR
│   └── >L KL/ <DH/ BN/ JFR>L/
│       └── GR
├── 3ms
│   └── NPC
│       └── 3ms
├── MCH=
├── 2ms
├── R<= -2ms
├── <MJT -2ms
├── KHN
├── BT -2ms
├── JD<NJ
├── MYRJM
├── >M/ ->JC W >B/ ->JC
├── >L H >LJL/
├── MN <M/ -NPC
├── XRC=/
├── PNH/ DL/
├── PNH/ GDWL/
├── B <MJT/ ->JC
├── L PNH/ <WR/
├── B <M/ -2ms
├── >T >X/ -2ms
├── >T BN/ <M/ -2ms
├── L >JC/
├── >L H >WB/
├── MN PNH/ FJBH/
└── PNH/ ZQN/


In [30]:
Lev19.resultRefs()

defaultdict(list,
            {'JHWH': [943680,
              943681,
              943683,
              943692,
              943693,
              943694,
              943695,
              63853,
              943702,
              943703,
              943704,
              943713,
              943714,
              943715,
              943720,
              943736,
              1317513,
              943775,
              943776,
              943777,
              63983,
              943796,
              943797,
              943820,
              943821,
              943843,
              943844,
              943867,
              943868,
              64086,
              943915,
              1317553,
              943964,
              943965,
              943966,
              943993,
              943994,
              64273,
              64276,
              944012,
              944013,
              944023,
              944024,
              944025,
         

In [31]:
error_list_Lev19 = Lev19.checkResults()
print(len(error_list_Lev19))

0


## Lev 20

In [32]:
Lev20 = Nodes.GenerateNodes('Leviticus',20)
Lev20.allRefs()
Lev20.AllRefs

{'JHWH': [944089,
  944090,
  944092,
  944112,
  944113,
  64439,
  944117,
  64461,
  64467,
  944142,
  944143,
  64497,
  944147,
  944163,
  64541,
  944167,
  944176,
  944177,
  944178,
  64565,
  944185,
  944186,
  944187,
  64818,
  64822,
  944371,
  944372,
  944382,
  944383,
  944389,
  944392,
  944398,
  944399,
  944404,
  944405,
  944406,
  944408,
  944427,
  64933,
  944435,
  944436,
  944437,
  944439,
  64947],
 'MCH=': [944091, 944095],
 'BN JFR>L': [944094,
  1317606,
  944171,
  944173,
  944174,
  64561,
  944180,
  944183,
  64571,
  64675,
  944273,
  944360,
  944363,
  64829,
  64835,
  944375,
  944379,
  64851,
  64862,
  944394,
  944395,
  64870,
  944401,
  64881,
  64884,
  944412,
  944420,
  64911,
  64928,
  944429,
  944431,
  944433,
  64941,
  944442],
 'JFR>L': [1317603, 1317605, 944101],
 '>JC': [944096,
  944097,
  944103,
  64422,
  944107,
  64431,
  944115,
  64447,
  64450,
  64453,
  944122,
  944124,
  944127,
  944135,
  944136,
  6

* NDH: Refers to a woman who is menstruating
* 3mp#3: Like other similar references, refer to two or more participants grouped together. Part-whole relationships
* KL#2: Refers to creeping animals so is not included
* '>T NPC/ -BN JFR>L': Refers to the soul of the Israelites. The reference is included because "soul" is deeply personal.
* Is GWJ "nations" and <M "people" synonymous? >M seems to refer to a broader category of which GWJ is one particular people.
* '>JC >CH "a man or a woman": Generic like >JC
* FRP refers to a third person masculine verb (probably synonoymous with <M ARY

In [33]:
humanRefs_Lev20 = ['JHWH','MCH=','BN JFR>L','>JC','GR','ZR< ->JC','MLK=','<M >RY','KL','NPC','>WB JD<NJ','>B ->JC >M ->JC',
                  '>B ->JC','>M ->JC','N>P N>P','CNJM -','CNJM -#2','>CH','CNJM -#3','>CH >M ->CH',
                  '2ms','>CH BHMH','BT >B ->JC BT >M ->JC','3unknownp','>CH#2','CNJM -#4','3ms','3mp','3mp#2','>X ->JC',
                  'NDH','3mp#3','GWJ','<M','>JC >CH','MN BN/ JFR>L/ W MN H GR/','MN QRB/ <M/ ->JC','>T CM/ QDC/ -JHWH',
                   'MN QRB/ <M/ -KL','MN QRB/ <M/ -NPC',
                   'B H >JC/ H HW> W B MCPXT/ ->JC','W >T KL/',
                   '>XR/ ->JC','>XR/ ->WB JD<NJ','>T >CH/ >JC/','>T >CH/ R<=/ ->JC','>T== >CH/ >B/ ->JC','<RWH/ >B/ ->JC',
                   '>T== KLH/ ->JC','>T== ZKR=/','FRP[','W','>T <RWH/ -BT >B ->JC BT >M ->JC','>T <RWH/ ->JC',
                   '<RWH/ >XWT/ ->JC','>T <RWH/ ->CH','MN QRB/ <M/ -CNJM -','<RWH/ >XWT/ >M/ -2msW >XWT/ >B/ -2ms',
                   '<RWH/ DWD/ ->JC','<RWH/ >X/ ->JC','>T== DWDH/ ->JC','>T >CH/ >X/ ->JC','MN PNH/ -BN JFR>L',
                   '>T NPC/ -BN JFR>L']

land_references_Lev20 = ['JFR>L','>RY','>DMH -GWJ','>RY/ ZWB[ XLB/ W DBC/']

In [34]:
synonyms_Lev20 = {'JHWH':['>T CM/ QDC/ -JHWH'],'>JC': ['NPC','MN BN/ JFR>L/ W MN H GR/','>XR/ ->JC','>T <RWH/ ->JC', '3ms'],
                  'BN JFR>L':['MN PNH/ -BN JFR>L','>T NPC/ -BN JFR>L'],'<M >RY':['FRP['],
                  'MN QRB/ <M/ ->JC':['MN QRB/ <M/ -NPC'],
                  '>T== >CH/ >B/ ->JC':['<RWH/ >B/ ->JC'],
                  'BT >B ->JC BT >M ->JC':['<RWH/ >XWT/ ->JC','>T <RWH/ -BT >B ->JC BT >M ->JC'],
                  '>T >CH/ >X/ ->JC':['NDH'],'>WB JD<NJ':['>XR/ ->WB JD<NJ'],'>T >CH/ >JC/':['>T >CH/ R<=/ ->JC'],
                 '>CH#2': ['>T <RWH/ ->CH'], '>T== DWDH/ ->JC': ['<RWH/ DWD/ ->JC'], '>T >CH/ >X/ ->JC': ['<RWH/ >X/ ->JC']}

hyponyms_Lev20 = {'W >T KL/':['>JC','KL'],'>B ->JC >M ->JC':['>B ->JC','>M ->JC'],'N>P N>P':['>JC','>T >CH/ >JC/'],
                  'CNJM -':['>JC','>T== >CH/ >B/ ->JC'],'CNJM -#2':['>JC','>T== KLH/ ->JC'],'CNJM -#3':['>JC','>T== ZKR=/'],
                  'CNJM -#4':['>JC','>CH#2'],'3unknownp':['>JC','BT >B ->JC BT >M ->JC'],
                  '3mp':['>JC', '<RWH/ >XWT/ >M/ -2msW >XWT/ >B/ -2ms'],
                  '3mp#2':['>JC','>T== DWDH/ ->JC'],'>CH BHMH':['>CH'],'3mp#3':['>JC','>T >CH/ >X/ ->JC'],'<M':['GWJ'],
                  'B H >JC/ H HW> W B MCPXT/ ->JC':['>JC'],'W':['>JC','>CH >M ->CH']}

In [35]:
show('MN QRB/ <M/ -CNJM -',Lev20.AllRefs,944353)

In [36]:
tree = Lev20.tree(humanRefs_Lev20, synonyms_Lev20, hyponyms_Lev20)
Lev20.treeStructure()
createPNG(tree, 'Leviticus', 20)

participants
├── W >T KL/
│   ├── >JC
│   │   ├── NPC
│   │   │   └── >JC
│   │   ├── MN BN/ JFR>L/ W MN H GR/
│   │   │   └── >JC
│   │   ├── >XR/ ->JC
│   │   │   └── >JC
│   │   ├── >T <RWH/ ->JC
│   │   │   └── >JC
│   │   └── 3ms
│   │       └── >JC
│   └── KL
├── >B ->JC >M ->JC
│   ├── >B ->JC
│   └── >M ->JC
├── N>P N>P
│   ├── >JC
│   │   ├── NPC
│   │   │   └── >JC
│   │   ├── MN BN/ JFR>L/ W MN H GR/
│   │   │   └── >JC
│   │   ├── >XR/ ->JC
│   │   │   └── >JC
│   │   ├── >T <RWH/ ->JC
│   │   │   └── >JC
│   │   └── 3ms
│   │       └── >JC
│   └── >T >CH/ >JC/
│       └── >T >CH/ R<=/ ->JC
│           └── >T >CH/ >JC/
├── CNJM -
│   ├── >JC
│   │   ├── NPC
│   │   │   └── >JC
│   │   ├── MN BN/ JFR>L/ W MN H GR/
│   │   │   └── >JC
│   │   ├── >XR/ ->JC
│   │   │   └── >JC
│   │   ├── >T <RWH/ ->JC
│   │   │   └── >JC
│   │   └── 3ms
│   │       └── >JC
│   └── >T== >CH/ >B/ ->JC
│       └── <RWH/ >B/ ->JC
│           └── >T== >CH/ >B/ ->JC
├── CNJM -#2
│   ├── >JC
│   │  

In [37]:
Lev20.resultRefs()

defaultdict(list,
            {'JHWH': [944089,
              944090,
              944092,
              944112,
              944113,
              64439,
              944117,
              64461,
              64467,
              944142,
              944143,
              64497,
              944147,
              944163,
              64541,
              944167,
              944176,
              944177,
              944178,
              64565,
              944185,
              944186,
              944187,
              64818,
              64822,
              944371,
              944372,
              944382,
              944383,
              944389,
              944392,
              944398,
              944399,
              944404,
              944405,
              944406,
              944408,
              944427,
              64933,
              944435,
              944436,
              944437,
              944439,
              64947,
              94

In [38]:
error_list_Lev20 = Lev20.checkResults()
print(len(error_list_Lev20))

0


## Lev 21

In [39]:
Lev21 = Nodes.GenerateNodes('Leviticus',21)
Lev21.allRefs()
Lev21.AllRefs

{'JHWH': [944458,
  944459,
  1317733,
  944545,
  944546,
  944547,
  944548,
  944598,
  944599,
  944621,
  944622,
  944623,
  944625,
  944626,
  944628,
  1317799,
  65338,
  944696,
  944697,
  944698],
 'MCH=': [944460, 944461, 944465, 944627, 944629, 944631, 944700, 944701],
 'BN >HRN': [944462,
  944463,
  64980,
  944495,
  65034,
  65037,
  944501,
  65042,
  944505,
  944507,
  944508,
  65049,
  944512,
  65054,
  65060,
  944517,
  944518,
  944520,
  944521,
  944524,
  944530],
 'HW>': [944469,
  64986,
  64990,
  64993,
  64995,
  64998,
  65001,
  65004,
  65007,
  65010,
  65015,
  944488,
  944490,
  65027,
  944493,
  65029,
  944532,
  944533,
  65083,
  65085,
  944539,
  944540,
  944541,
  944542],
 '<M -HW>': [944470, 944492],
 'C>R -HW>': [944472, 944474],
 'BTWLH': [944478, 944479, 944481, 944485, 944486, 65021],
 '>LHJM -BN >HRN': [944509, 1317731, 1317735],
 'LXM >LHJM -BN >HRN': [944515, 944516, 944671],
 '>CH': [944526, 944527, 65076],
 '2ms': [944536, 

* 'L NPC/' refers to a dead body but may still be seen as a reference to a generic person.
* 'CM/ >LHJM/ -BN >HRN' refers to the name of God, essentially God himself.
* 'C>R -HW>' "his closest relatives" is technically a part of <M -HW> "his kinsmen", but in the text the two entites are rather contrasted rather than derived. Therefore not hyponyms.
* '>T >B/ -BT >JC KHN' "The father of any priest's daugther" seems to be synonymous to "priest", that is, 'BN >HRN'
* BTWLH/ is synonymous with '>CH/' "wife". '>CH/', however, is not really a participant but a title.
* '>JC#2' is specific man who is either blind, lame or mutilated, synonymous to 'KL/ >JC/'

In [40]:
humanRefs_Lev21 = ['JHWH','MCH=','BN >HRN','HW>','<M -HW>','C>R -HW>','BTWLH','>LHJM -BN >HRN','>CH','2ms','BT >JC KHN',
                  'KHN','>LHJM -KHN','>CH#2','>LMNH GRC XLL','<M -KHN','>HRN','>JC','DWR ->JC','>LHJM ->JC','>JC#2','L NPC/',
                  'L >M/ -HW>W L >B/ -HW>W L BN/ -HW>W L BT/ -HW>W L >X/ -HW>','CM/ >LHJM/ -BN >HRN','>CH/ ZNH[ W XLL/',
                  'MN >JC/ ->CH','L >LHJM/ -HW>','2mp_sfx','>T >B/ -BT >JC KHN','MN >X/ -KHN','BTWLH/','<L KL/ NPC/ MWT[',
                   'L >B/ -KHN W L >M/ -KHN','ZR</ -KHN',
                   'KL/ >JC/','KL/ >JC/#2','>L >HRN/ W >L BN/ ->HRN W >L KL/ BN/ JFR>L/']

land_references_Lev21 = []

In [41]:
synonyms_Lev21 = {'JHWH':['>LHJM -BN >HRN','>LHJM -KHN','>LHJM ->JC','CM/ >LHJM/ -BN >HRN','L >LHJM/ -HW>'],
                 'BN >HRN':['HW>', '>T >B/ -BT >JC KHN'],'BTWLH/': ['>CH#2'],'>LMNH GRC XLL': ['>CH'],
                  'C>R -HW>':['L >M/ -HW>W L >B/ -HW>W L BN/ -HW>W L BT/ -HW>W L >X/ -HW>','BTWLH'],
                 '>JC':['DWR ->JC','KL/ >JC/#2'], 'KL/ >JC/':['>JC#2'], 'L NPC/': ['<L KL/ NPC/ MWT[']}

hyponyms_Lev21 = {'>L >HRN/ W >L BN/ ->HRN W >L KL/ BN/ JFR>L/':['>HRN','BN >HRN']}

In [42]:
show('KL/ >JC/',Lev21.AllRefs)

In [43]:
tree = Lev21.tree(humanRefs_Lev21, synonyms_Lev21, hyponyms_Lev21)

In [44]:
createPNG(tree, 'Leviticus', 21)

In [45]:
Lev21.resultRefs()

defaultdict(list,
            {'JHWH': [944458,
              944459,
              1317733,
              944545,
              944546,
              944547,
              944548,
              944598,
              944599,
              944621,
              944622,
              944623,
              944625,
              944626,
              944628,
              1317799,
              65338,
              944696,
              944697,
              944698,
              944509,
              1317731,
              1317735,
              1317755,
              1317759,
              1317767,
              1317801,
              1317803,
              944513,
              944534],
             'MCH=': [944460,
              944461,
              944465,
              944627,
              944629,
              944631,
              944700,
              944701],
             'BN >HRN': [944462,
              944463,
              64980,
              944495,
              65034,
 

In [46]:
error_list_Lev21 = Lev21.checkResults()
len(error_list_Lev21)

0

## Lev 22

In [47]:
Lev22 = Nodes.GenerateNodes('Leviticus',22)
Lev22.allRefs()
Lev22.AllRefs

{'JHWH': [944704,
  944705,
  944707,
  65380,
  65384,
  944721,
  944722,
  944734,
  65418,
  944742,
  944743,
  944812,
  944813,
  65522,
  944827,
  944828,
  944829,
  944902,
  944910,
  944911,
  944912,
  944914,
  944915,
  944917,
  944932,
  944953,
  944970,
  944977,
  944991,
  945011,
  945012,
  945014,
  945028,
  945039,
  945048,
  945049,
  65908,
  945056,
  945057,
  65919,
  945063,
  945065,
  945066,
  945067,
  945069,
  945072,
  945074,
  945075,
  945076],
 'MCH=': [944706, 944708, 944723, 944916, 944918, 944921, 945013],
 '>HRN BN ->HRN': [944709,
  944711,
  944715,
  65388,
  65390,
  65397,
  944815,
  944819,
  944823,
  944826,
  65535,
  944898,
  944904,
  944907,
  65647,
  65653,
  1317884],
 '>HRN': [1317818, 65368, 1317837, 1317882, 65666],
 'BN ->HRN': [1317819, 1317883],
 'QDC BN JFR>L': [944712, 944899, 944908],
 'BN JFR>L': [1317823,
  944718,
  944719,
  944732,
  944733,
  1317879,
  944901,
  65643,
  65649,
  1317888,
  1317969],
 'JF

In [48]:
checkList(Lev22.AllRefs)

L DWR/ ->HRN BN ->HRN


* 'CM QDC -JHWH': Refers to the name of YHWH and therefore to YHWH himself.
* '>JC' refers to the offspring of both Aaron and his sons (synonymous to NPC)
* '>JC >JC' refers to the offspring of Aaron alone (synonymous to NPC#2)
* '>JC#2' is a lay person
* '>DM' is generic (cf. v. 5)
* '>JC >JC#2' is generic, either Israelite of sojourner.

In [49]:
humanRefs_Lev22 = ['JHWH','MCH=','>HRN BN ->HRN','>HRN','BN ->HRN','BN JFR>L','CM QDC -JHWH','>JC','NPC','>JC >JC','>JC#2',
                   '>DM','NPC#2','NPC#3','MN L PNH/ -JHWH',
                  'ZR','TWCB KHN FKJR','KHN','JLJD BJT -KHN','BT KHN','>B -BT KHN','>HRN BN ->HRN KL BN JFR>L','>JC >JC#2',
                  'L DWR/ ->HRN BN ->HRN','MN ZR</ >HRN/','B KL/ VM>/ NPC/','KL/ ZR/','ZR</','KL/ ZR/#2','B TWK/ BN/ JFR>L/']

land_references_Lev22 = ['JFR>L']

In [50]:
synonyms_Lev22 = {'JHWH':['CM QDC -JHWH','MN L PNH/ -JHWH'],'>JC':['NPC'],
                  'MN ZR</ >HRN/':['>JC >JC','KHN','>B -BT KHN','NPC#2'],
                 'ZR':['KL/ ZR/','KL/ ZR/#2','>JC#2'], 'BN JFR>L':['B TWK/ BN/ JFR>L/']}

hyponyms_Lev22 = {'>HRN BN ->HRN KL BN JFR>L': ['>HRN BN ->HRN','BN JFR>L'], '>HRN BN ->HRN':['>HRN','BN ->HRN']}

In [51]:
show('>DM',Lev22.AllRefs, 944526)

In [52]:
tree = Lev22.tree(humanRefs_Lev22, synonyms_Lev22, hyponyms_Lev22)
createPNG(tree, 'Leviticus', 22)

In [53]:
Lev22.resultRefs()

defaultdict(list,
            {'JHWH': [944704,
              944705,
              944707,
              65380,
              65384,
              944721,
              944722,
              944734,
              65418,
              944742,
              944743,
              944812,
              944813,
              65522,
              944827,
              944828,
              944829,
              944902,
              944910,
              944911,
              944912,
              944914,
              944915,
              944917,
              944932,
              944953,
              944970,
              944977,
              944991,
              945011,
              945012,
              945014,
              945028,
              945039,
              945048,
              945049,
              65908,
              945056,
              945057,
              65919,
              945063,
              945065,
              945066,
              945067,
            

In [54]:
error_list_Lev22 = Lev22.checkResults()
print(len(error_list_Lev22))

0


## Lev 23

In [55]:
Lev23 = Nodes.GenerateNodes('Leviticus',23)
Lev23.allRefs()
Lev23.AllRefs

{'JHWH': [945078,
  945079,
  945081,
  1317975,
  65965,
  945107,
  1317991,
  945121,
  945126,
  945140,
  945148,
  945149,
  945151,
  945161,
  945162,
  1318033,
  945187,
  945194,
  945224,
  945233,
  945247,
  945251,
  1318117,
  945270,
  945298,
  945299,
  945302,
  945303,
  945305,
  945323,
  945325,
  945326,
  945328,
  945342,
  1318173,
  945354,
  945371,
  945394,
  945395,
  945397,
  945405,
  945414,
  945422,
  1318233,
  945436,
  1318246,
  945443,
  1318273,
  1318299,
  945465,
  945471,
  945491,
  945493,
  66758,
  945496,
  945497,
  945498,
  1318317],
 'MCH=': [945080,
  945082,
  945085,
  945150,
  945152,
  945155,
  945304,
  945306,
  945308,
  945327,
  945396,
  945398,
  945400,
  945500,
  945501],
 'BN JFR>L': [945083,
  65955,
  945089,
  945104,
  65990,
  945113,
  945129,
  66045,
  945136,
  945138,
  945146,
  945153,
  66083,
  945158,
  66092,
  945165,
  945168,
  66102,
  66115,
  945180,
  945182,
  66127,
  945202,
  945204,


* 'NPC -BN JFR>L' refers to the soul of the Israelites, therefore (almost) synonymous to BN JFR>L.
* 'NPC' is generic. Seems to exemplify prohibitions related to BN JFR>L. '>ZRX' is also generic so is treated as a synonym to NPC.

In [56]:
humanRefs_Lev23 = ['JHWH','MCH=','BN JFR>L','KHN','PNH JHWH','>LHJM -BN JFR>L','DWR -BN JFR>L','2ms','NPC -BN JFR>L','NPC',
                  '>ZRX','L H <NJ/ W L H GR/','MN <M/ -NPC','MN QRB/ <M/ -NPC','KL/ H >ZRX/']

land_references_Lev23 = ['JFR>L','>RY','MN >RY/ MYRJM/']

In [57]:
synonyms_Lev23 = {'JHWH':['PNH JHWH','>LHJM -BN JFR>L'], 'BN JFR>L': ['NPC -BN JFR>L'], '>ZRX':['KL/ H >ZRX/'],
                 'MN <M/ -NPC':['MN QRB/ <M/ -NPC']}

hyponyms_Lev23 = {}

In [58]:
#show('MN <M/ -NPC',Lev23.AllRefs)

In [59]:
tree = Lev23.tree(humanRefs_Lev23, synonyms_Lev23, hyponyms_Lev23)
createPNG(tree, 'Leviticus', 23)

In [60]:
Lev23.resultRefs()

defaultdict(list,
            {'JHWH': [945078,
              945079,
              945081,
              1317975,
              65965,
              945107,
              1317991,
              945121,
              945126,
              945140,
              945148,
              945149,
              945151,
              945161,
              945162,
              1318033,
              945187,
              945194,
              945224,
              945233,
              945247,
              945251,
              1318117,
              945270,
              945298,
              945299,
              945302,
              945303,
              945305,
              945323,
              945325,
              945326,
              945328,
              945342,
              1318173,
              945354,
              945371,
              945394,
              945395,
              945397,
              945405,
              945414,
              945422,
              1318233,
 

In [61]:
error_list_Lev23 = Lev23.checkResults()
print(len(error_list_Lev23))

0


## Lev 24

In [62]:
Lev24 = Nodes.GenerateNodes('Leviticus',24)
Lev24.allRefs()
Lev24.AllRefs

{'JHWH': [945505,
  945506,
  945508,
  1318333,
  1318339,
  1318353,
  1318361,
  1318373,
  1318405,
  945616,
  945617,
  945619,
  1318415,
  945710,
  945711,
  945712,
  945730,
  945731],
 'MCH=': [945507,
  945509,
  66788,
  945537,
  945540,
  945547,
  945555,
  945603,
  945618,
  945620,
  945635,
  945636,
  945714,
  945715,
  945732],
 'BN JFR>L': [945510,
  945512,
  66823,
  945569,
  945589,
  945634,
  67119,
  67130,
  945716,
  945718,
  945723,
  945727,
  945728],
 'JFR>L': [1318321, 1318363, 1318386, 1318411, 1318427, 1318429],
 'CMN ZJT': [945514, 945516],
 'NR': [945517, 66810],
 'XWY': [945519, 945622, 945720],
 '>HRN': [945522, 945524, 945532, 945566, 1318366, 66911, 66921],
 'PNH JHWH': [945527, 945534, 945553, 945567],
 '<WLM': [1318335, 1318365, 1318375],
 'SLT': [945538, 66842],
 'CNJM <FRH XLH': [945542, 66855],
 'CNJM <FRWN': [945543, 945544, 945545],
 'CC': [945550, 945551, 945556],
 'LBNH=': [945557, 945559],
 'QDC QDC': [945560, 66894, 66913, 9455

* '3mp' refers to a particular group of witnesses (cf. v. 14)
* '<DH' "congregation" likely refers to BN JFR>L (cf. v. 23)
* Both >JC and >JC >JC are generic and therefore synonymous
* '>DM' is also generic but is distinct from >JC and >JC >JC.
* '>KL[ -QDC QDC' refers to Aaron and his sons (v. 9)
* '<L R>C/ -QLL' "on the head of the curser" refers to the curser.

In [63]:
humanRefs_Lev24 = ['JHWH','MCH=','BN JFR>L','>HRN','PNH JHWH','>HRN BN ->HRN','BN >CH','>CH','BN JFR>LJ >JC JFR>LJ',
                   'CM JHWH','3mp','CLMJT','QLL','CM<','<DH','>JC >JC','GR','>JC','>DM','L DWR/ -BN JFR>L','KL/ H CM<[',
                  '<L R>C/ -QLL','KL/ H <DH/','>LHJM/ ->JC >JC','KL/ H <DH/#2','KL/ NPC/ >DM/','B <MJT/ ->JC']

land_references_Lev24 = ['JFR>L']

In [64]:
synonyms_Lev24 = {'JHWH':['PNH JHWH','CM JHWH','>LHJM/ ->JC >JC'], '3mp':['CM<','KL/ H CM<['], 'CLMJT':['>CH'],
                  'BN >CH':['QLL','<L R>C/ -QLL'], 'BN JFR>L':['<DH','KL/ H <DH/','KL/ H <DH/#2'], '>JC >JC':['>JC'],
                 '>DM': ['KL/ NPC/ >DM/']}

hyponyms_Lev24 = {'BN JFR>LJ >JC JFR>LJ':['BN >CH'], '>HRN BN ->HRN':['>HRN']}

In [65]:
#show('>HRN BN ->HRN',Lev24.AllRefs)

In [66]:
tree = Lev24.tree(humanRefs_Lev24, synonyms_Lev24, hyponyms_Lev24)
createPNG(tree, 'Leviticus', 24)

In [67]:
Lev24.resultRefs()

defaultdict(list,
            {'JHWH': [945505,
              945506,
              945508,
              1318333,
              1318339,
              1318353,
              1318361,
              1318373,
              1318405,
              945616,
              945617,
              945619,
              1318415,
              945710,
              945711,
              945712,
              945730,
              945731,
              945527,
              945534,
              945553,
              945567,
              945597,
              945646,
              945656,
              945640],
             'MCH=': [945507,
              945509,
              66788,
              945537,
              945540,
              945547,
              945555,
              945603,
              945618,
              945620,
              945635,
              945636,
              945714,
              945715,
              945732],
             'BN JFR>L': [945510,
              945512,


In [68]:
error_list_Lev24 = Lev24.checkResults()
print(len(error_list_Lev24))

0


## Lev 25

In [69]:
Lev25 = Nodes.GenerateNodes('Leviticus',25)
Lev25.allRefs()
Lev25.AllRefs

{'JHWH': [945734,
  945735,
  945738,
  945748,
  945749,
  945755,
  945917,
  945918,
  945919,
  67474,
  67477,
  945958,
  67524,
  67571,
  67579,
  946172,
  946173,
  946174,
  946176,
  946179,
  946182,
  946184,
  67905,
  946219,
  68132,
  68136,
  946374,
  946377,
  946378,
  946379],
 'MCH=': [945736, 945739, 945742],
 'GR TWCB': [945740,
  67178,
  945745,
  67187,
  67252,
  945824,
  67331,
  945828,
  945832,
  67351,
  945841,
  945847,
  67369,
  945855,
  945858,
  945862,
  67387,
  945871,
  945874,
  945879,
  945883,
  945886,
  945910,
  67470,
  945921,
  945925,
  945927,
  945930,
  945938,
  945941,
  945946,
  945948,
  945952,
  945955,
  67520,
  67525,
  945967,
  945970,
  945976,
  945987,
  945988,
  67584,
  945993,
  1318575,
  67844,
  67847,
  67853,
  67859,
  67938,
  946241,
  67952,
  946249,
  67959,
  67963,
  67966,
  946262,
  67973,
  67974,
  946270,
  67984,
  946273,
  946274,
  67989,
  68146],
 'JFR>L': [1318433, 1318574, 1318611

* GR TWCB refers to BN JFR>L
* '>JC' is generic. Sometimes synonymous to "your brother"
* TWCB specifies GR, therefore synonymous.
* Are >X -2ms and <MJT -2ms synonymous? Probably (cf. v. 14)
* GR TWCB#2 refers to the status of "your brother", therefore synonymous.
* '<BD -JHWH' refers to the BN JFR>L (= GR TWCB)
* 'QNH ->X -2ms' refers to the stranger (GR#2)
* 'L KL/ JCB[ ->RY' "all inhabitants of the land": Probably refers to both the Israelites and the so-journers.
* '3ms' seems to be "one of the Levites" (v. 33)

In [70]:
humanRefs_Lev25 = ['JHWH','MCH=','GR TWCB','2ms','>MH -2ms','>JC','<MJT -2ms','>LHJM -2ms','GR','TWCB','>X -2ms',
                  'G>L ->X -2ms','G>L','JD ->JC','>JC#2','QNH','LWJ','3ms','JD ->X -2ms','HW>BN ->X -2ms',
                   'BN ->X -2ms','<BD -JHWH','<BD -2ms>MH -2ms','SBJB -GR TWCB','BN TWCB','BN -GR TWCB','>X -GR TWCB',
                   'JD GR TWCB','GR#2','>XD','DWD ->X -2msBN DWD ->X -2ms','DWD ->X -2ms','QNH ->X -2ms',
                  'W L <BD/ -2msW L >MH/ -2msW L FKJR/ -2msW L TWCB/ -2ms','GWR[','L KL/ JCB[ ->RY','>L MCPXT/ ->JC',
                  'MN JD/ <MJT/ -2ms','>T >X/ ->JC','>T <MJT/ ->JC','<MD/ -JHWH','B TWK/ BN/ JFR>L/',
                   'MN','MN#2','>L MCPXT/ ->X -2ms','MN MCPXT/ -BN TWCB','>XR/ -GR TWCB','L <QR=/ MCPXT/ GR/']

land_references_Lev25 = ['>RY','>RY MYRJM','B KL/ >RY/ -GR TWCB','>T >RY/ KN<N/']

In [71]:
synonyms_Lev25 = {'JHWH':['>LHJM -2ms','<MD/ -JHWH'],'2ms':['MN#2'],'>JC':['JD ->JC'],
                  '>X -2ms': ['<MJT -2ms','MN JD/ <MJT/ -2ms','MN','JD ->X -2ms'], 'LWJ':['3ms'],
                  'GR TWCB':['GR','TWCB','<BD -JHWH','>XR/ -GR TWCB','B TWK/ BN/ JFR>L/'],
                  '>T >X/ ->JC':['>T <MJT/ ->JC'],'GR#2':['JD GR TWCB','QNH ->X -2ms']}

hyponyms_Lev25 = {'HW>BN ->X -2ms':['>X -2ms','BN ->X -2ms'],
                  'W L <BD/ -2msW L >MH/ -2msW L FKJR/ -2msW L TWCB/ -2ms':['2ms','<BD -2ms>MH -2ms','GWR['],
                  '<BD -2ms>MH -2ms':['>MH -2ms'], 'DWD ->X -2msBN DWD ->X -2ms':['DWD ->X -2ms']}

In [72]:
show('GR TWCB',Lev25.AllRefs)

In [73]:
tree = Lev25.tree(humanRefs_Lev25, synonyms_Lev25, hyponyms_Lev25)
createPNG(tree, 'Leviticus', 25)

In [74]:
Lev25.resultRefs()

defaultdict(list,
            {'JHWH': [945734,
              945735,
              945738,
              945748,
              945749,
              945755,
              945917,
              945918,
              945919,
              67474,
              67477,
              945958,
              67524,
              67571,
              67579,
              946172,
              946173,
              946174,
              946176,
              946179,
              946182,
              946184,
              67905,
              946219,
              68132,
              68136,
              946374,
              946377,
              946378,
              946379,
              945915,
              946157,
              946231,
              945989],
             'MCH=': [945736, 945739, 945742],
             'GR TWCB': [945740,
              67178,
              945745,
              67187,
              67252,
              945824,
              67331,
              945828,
   

In [75]:
error_list_Lev25 = Lev25.checkResults()
print(len(error_list_Lev25))

0


## Lev 26

In [76]:
Lev26 = Nodes.GenerateNodes('Leviticus',26)
Lev26.allRefs()
Lev26.AllRefs

{'<M': [946381,
  68149,
  946387,
  68157,
  946392,
  68164,
  946394,
  68171,
  946401,
  946404,
  946409,
  946412,
  946414,
  68193,
  68209,
  946438,
  68220,
  946442,
  68228,
  946450,
  68252,
  946464,
  68256,
  68260,
  68266,
  68271,
  68276,
  68278,
  68284,
  68287,
  68290,
  68295,
  946499,
  946504,
  68310,
  68315,
  68319,
  68322,
  946522,
  946523,
  946525,
  68333,
  68336,
  946533,
  68347,
  68350,
  946546,
  946550,
  946555,
  68376,
  946561,
  946563,
  68384,
  68391,
  68394,
  946580,
  68413,
  68416,
  68420,
  946591,
  68425,
  68428,
  68429,
  946598,
  68435,
  946607,
  68447,
  68450,
  68455,
  68459,
  68465,
  68474,
  68478,
  946641,
  946646,
  946647,
  68501,
  68505,
  68508,
  68515,
  68519,
  68522,
  68525,
  946675,
  946678,
  68541,
  68546,
  68551,
  68554,
  946699,
  68562,
  68567,
  946706,
  68575,
  68582,
  68588,
  946721,
  946724,
  946729,
  946732,
  68611,
  68617,
  68622,
  946746,
  68626,
  68629,


* '<M' refers to BN JFR>L in other chapters
* GWJ seems to be synonymous with the enemies of the people.
* 'HM' refers to a remnant of the Israelites.
* '>JC' is generic - a generic person within the group of HM (the remnant)

In [77]:
humanRefs_Lev26 = ['<M','JHWH','>JB -<M','M>H','NPC -JHWH','MYRJM','NPC -<M','L PNH/ >JB[ -<M','FN> -<M','RDP',
                  '<FR= >CH','GWJ','HM','LBB -HM','>JB -HM','>JC','>B -HM','NPC -HM','L PNH/ -<M','XMC','M>H/','RBBH/',
                  'L PNH/ -<M#2','B TWK/ -<M','B TWK/ -<M#2','<BD/','<JN/','NPC/','PNH/ -JHWH','B TWK/ -<M#3','B JD/ >JB[',
                  'BFR/ BN/ -<M','BFR/ BT/ -<M','>XR/ -<M','B >X/ ->JC','J<QB/','JYXQ/','>BRHM/',
                  'BJN/ -JHWH W BJN/ BN/ JFR>L/','>T PGR/ -<M']

land_references_Lev26 = ['>RY -<M','>RY','>RY MYRJM','>RY >JB -HM','>RY >JB -<M']

In [78]:
synonyms_Lev26 = {'<M':['M>H','NPC -<M','L PNH/ -<M','XMC','L PNH/ -<M#2','B TWK/ -<M','B TWK/ -<M#2','<BD/','<JN/','NPC/',
                       'B TWK/ -<M#3','>XR/ -<M'],
                  'JHWH':['NPC -JHWH','PNH/ -JHWH'],
                  '>JB -<M':['L PNH/ >JB[ -<M','FN> -<M','GWJ','M>H/','RBBH/','B JD/ >JB['],
                 'HM':['LBB -HM','NPC -HM']}

hyponyms_Lev26 = {'BJN/ -JHWH W BJN/ BN/ JFR>L/':['JHWH','<M']}

In [79]:
show('L PNH/ >JB[ -<M',Lev26.AllRefs)

In [80]:
tree = Lev26.tree(humanRefs_Lev26, synonyms_Lev26, hyponyms_Lev26)
createPNG(tree, 'Leviticus', 26)

In [81]:
Lev26.resultRefs()

defaultdict(list,
            {'<M': [946381,
              68149,
              946387,
              68157,
              946392,
              68164,
              946394,
              68171,
              946401,
              946404,
              946409,
              946412,
              946414,
              68193,
              68209,
              946438,
              68220,
              946442,
              68228,
              946450,
              68252,
              946464,
              68256,
              68260,
              68266,
              68271,
              68276,
              68278,
              68284,
              68287,
              68290,
              68295,
              946499,
              946504,
              68310,
              68315,
              68319,
              68322,
              946522,
              946523,
              946525,
              68333,
              68336,
              946533,
              68347,
            

In [82]:
error_list_Lev26 = Lev26.checkResults()
print(len(error_list_Lev26))

0


## Export nodes

In [83]:
nodes_to_export = [Lev17.resultRefs(), Lev18.resultRefs(), Lev19.resultRefs(), Lev20.resultRefs(), Lev21.resultRefs(),
                   Lev22.resultRefs(), Lev23.resultRefs(), Lev24.resultRefs(), Lev25.resultRefs(), Lev26.resultRefs()]

In [84]:
file_name = 'human_references_for_plotting.csv'
header = f'book,chapter,actor,references'

with open(file_name, 'w') as f:
    f.write('''{}'''.format(header))
    for chap in nodes_to_export:
        for actor in chap:
            bo, ch = T.sectionFromNode(chap[actor][0])[:2]
            references = ' '.join(str(e) for e in chap[actor])
            f.write('''\n{},{},{},{}'''.format(bo, ch, actor, references))